# Topics in Computer Science - Bitcoin: Programming the Future of Money - ITCS 4010 & 5010 - Spring 2025 - UNC Charlotte

# Reading Quiz 5 - How to derive a P2PKH address from Private/Public Keys


The following classes `FieldElement` and `Point` implement instances of elements of a finite field and of points on a elliptic curve, and their respective arithmetic operations. You might be familiar with them from the last homework assignment.

In [1]:
class FieldElement:

    def __init__(self, num, prime):
        #check if 0 > num >= prime. Raise ValueError if num is out of range.
        if num >= prime or num < 0:
            error = 'Num {} not in field range 0 to {}'.format(num, prime - 1)
            raise ValueError(error)
        #Initialize num and prime
        self.num = num
        self.prime = prime

    def __repr__(self):
        return 'FieldElement_{}({})'.format(self.prime, self.num)

    def __eq__(self, other):
        if other is None:
            return False
        #Return True if the FieldElement objects are equal
        return self.num == other.num and self.prime == other.prime

    def __ne__(self, other):
        # This should be the inverse of the == operator
        return not (self == other)

    def __add__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot add two numbers in different Fields')
        #Perform addition of two finite field elements
        num = (self.num + other.num) % self.prime
        # Return an element of the same class
        return self.__class__(num, self.prime)

    def __sub__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot subtract two numbers in different Fields')
        #Perform subtraction of two finite field elements
        num = (self.num - other.num) % self.prime
        return self.__class__(num, self.prime)

    def __mul__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot multiply two numbers in different Fields')
        # Perform muliplication of two finite field elements
        num = (self.num * other.num) % self.prime
        return self.__class__(num, self.prime)

    def __pow__(self, exponent):
        #Implement finite field exponentation
        n = exponent % (self.prime - 1)
        num = pow(self.num, n, self.prime)
        return self.__class__(num, self.prime)

    def __truediv__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot divide two numbers in different Fields')
        # perform division of two finite field elements
        # Hint: Use fermat's little theorem:
        num = (self.num * pow(other.num, self.prime - 2, self.prime)) % self.prime
        return self.__class__(num, self.prime)

    def __rmul__(self, coefficient):
        # Implement scalar multiplication: Multiply the scalar 'coeffiecient' with finite field element.
        num = (self.num * coefficient) % self.prime
        return self.__class__(num=num, prime=self.prime)

In [2]:
class Point:

    def __init__(self, x, y, a, b):
        self.a = a
        self.b = b
        self.x = x
        self.y = y
        if self.x is None and self.y is None:
            return
        if self.y**2 != self.x**3 + a * x + b:
            # if not, throw a ValueError
            raise ValueError('({}, {}) is not on the curve'.format(x, y))

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y \
            and self.a == other.a and self.b == other.b

    def __ne__(self, other):
        # this should be the inverse of the == operator
        return not (self == other)

    def __repr__(self):
        if self.x is None:
            return 'Point(infinity)'
        elif isinstance(self.x, FieldElement):
            return 'Point({},{})_{}_{} FieldElement({})'.format(
                self.x.num, self.y.num, self.a.num, self.b.num, self.x.prime)
        else:
            return 'Point({},{})_{}_{}'.format(self.x, self.y, self.a, self.b)

    def __add__(self, other):
        if self.a != other.a or self.b != other.b:
            raise TypeError('Points {}, {} are not on the same curve'.format(self, other))
        
        if self.x is None:
            return other
        if other.x is None:
            return self

        if self.x == other.x and self.y != other.y:
            return self.__class__(None, None, self.a, self.b)

        if self.x != other.x:
            s = (other.y - self.y) / (other.x - self.x)
            x = s**2 - self.x - other.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)

        if self == other and self.y == 0 * self.x:
            return self.__class__(None, None, self.a, self.b)

        if self == other:
            s = (3 * self.x**2 + self.a) / (2 * self.y)
            x = s**2 - 2 * self.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)

    def __rmul__(self, coefficient):
        coef = coefficient
        current = self
        result = self.__class__(None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result


The classes `S256Field` and `S256Point` are subclasses of `FieldElement` and `Point`, respectively, specifically designed to work with the parameters of [secp256k1](https://en.bitcoin.it/wiki/Secp256k1). These subclasses simplify the process of initializing a point on the secp256k1 curve by eliminating the need to repeatedly define the curve parameters `a` and `b`, as required when using the Point class.

In [3]:
Prime = 2**256 - 2**32 - 977 # this is the prime that determines the size of the finite field F_p on which the secp256k1 points live
class S256Field(FieldElement):

    def __init__(self, num, prime=None):
        super().__init__(num=num, prime=Prime)

    def __repr__(self):
        return '{:x}'.format(self.num).zfill(64)

Besides specifying the prime order and the curve parameters `a` and `b`, secp256k1 also specifies a specific point on the elliptic curve, the so-called _generator point_ `G`. It is defined via its $x$- and $y$-coordinates $G=(G_x,G_y)$.

In [4]:
A = 0
B = 7
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141 # order of group generated by generator point G

class S256Point(Point):

    def __init__(self, x, y, a=None, b=None):
        a, b = S256Field(A), S256Field(B)
        if type(x) == int:
            super().__init__(x=S256Field(x), y=S256Field(y), a=a, b=b)
        else:
            super().__init__(x=x, y=y, a=a, b=b)  # <1>

    def __repr__(self):
        if self.x is None:
            return 'S256Point(infinity)'
        else:
            return 'S256Point({}, {})'.format(self.x, self.y)

    def __rmul__(self, coefficient):
        coef = coefficient % n
        return super().__rmul__(coef)

G = S256Point(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
    0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)

The group order $n$ of the group generated by $G$ corresponds to the smallest integer that satisfies 
$$
n \cdot G = O,
$$
(scalar multplication of $n$ with $G$), where $O$ is the additive identity element of the elliptic curve $S_{0,7} = \{(x,y) \in F_p \times F_p: y^2 = x^3 + 7\}$ with $p=$`Prime`. You can see in the definition of `__rmul__` above that this group order can be used to simplify a scalar multiplication coefficient `coefficient`; this is due to the fact that all calculations done within our signature schemes are within the generator group.

## Generation of Private Key

In [5]:
import random
random.seed(1337)
e = random.randint(0,n) # generation of private key
print("Private key (decimal integer representation): \n", e)

Private key (decimal integer representation): 
 84747631942840761409198475171043116002924132430274400095798688737583350222083


In [6]:
e_hex = hex(e)
print("Private key (hex representation): \n", e_hex)

Private key (hex representation): 
 0xbb5d75b895f628f2922badb05da83cffb5bab1cd888417a5ecefe37b9e250d03


## Computation of Public Key from Private Key

In [7]:
P = e*G

In [8]:
P

S256Point(e49a3e63ffb8649dc976bdf178bbc2a6035f5085f805fed8100ff017e4a397b2, cd3376eddb0561f70b82f11228f17d2809b9cd00f7dc86ca4a4f240bdd06582b)

### Derivation of Uncompressed SEC format of private key P
Calculate uncompressed SEC format of public key `P` above.

In [9]:
P_uncompressedSEC = P.y

In [10]:
P_uncompressedSEC = '04'+P.x.__repr__()+P.y.__repr__()
print("Uncompressed SEC format of public key: \n", P_uncompressedSEC)

Uncompressed SEC format of public key: 
 04e49a3e63ffb8649dc976bdf178bbc2a6035f5085f805fed8100ff017e4a397b2cd3376eddb0561f70b82f11228f17d2809b9cd00f7dc86ca4a4f240bdd06582b


### Derivation of Compressed SEC format of private key P
Calculate compressed SEC format of public key `P` above.

In [11]:
print('y-coordinate of public key P: \n',P.y.__repr__())

y-coordinate of public key P: 
 cd3376eddb0561f70b82f11228f17d2809b9cd00f7dc86ca4a4f240bdd06582b


The last byte is a `2b` in hex representation, which correspond to s to the decimal number $2*16^1+11*16^0$ which is an add number. Therefore, we prepend a `03` byte (otherwise, in case of an odd y-coordinate, it would be a `04` byte).

In [12]:
P_compressedSEC = '03'+P.x.__repr__()
print("Compressed SEC format of public key: \n",P_compressedSEC)

Compressed SEC format of public key: 
 03e49a3e63ffb8649dc976bdf178bbc2a6035f5085f805fed8100ff017e4a397b2


## Derivation of P2PKH address

Obtain the hash160 image of the above public key in compressed SEC format. The hash160 image is computed by applying first the sha256 hash function to the byte-wise serialization of the compressed SEC public key, and then the RIPEMD-160 hash function to the resulting image.

Finally, the result will be encoded using the base58check encoding, which corresponds then to the desired P2PKH address.

### Create Byte-representation of Compressed SEC public key

In [13]:
P_compressedSEC_str = bytes.fromhex(P_compressedSEC)
print(P_compressedSEC_str)

b'\x03\xe4\x9a>c\xff\xb8d\x9d\xc9v\xbd\xf1x\xbb\xc2\xa6\x03_P\x85\xf8\x05\xfe\xd8\x10\x0f\xf0\x17\xe4\xa3\x97\xb2'


### Compute SHA-256 image

In [14]:
import hashlib

In [15]:
Sha256hash_P_bytes = hashlib.sha256(P_compressedSEC_str).digest()
Sha256hash_P_hex = hashlib.sha256(P_compressedSEC_str).hexdigest()
print("Hex representation of SHA-256 image: \n",Sha256hash_P_hex)
print("Bytes of SHA-256 image: \n", Sha256hash_P_bytes)

Hex representation of SHA-256 image: 
 89e15acf1714db44d995d9bd5c8fef69afef4931e6cee18b29c803dc428999fb
Bytes of SHA-256 image: 
 b'\x89\xe1Z\xcf\x17\x14\xdbD\xd9\x95\xd9\xbd\\\x8f\xefi\xaf\xefI1\xe6\xce\xe1\x8b)\xc8\x03\xdcB\x89\x99\xfb'


### Compute RIPEMD-160 image of SHA-256 image

In [16]:
ripmd160 = hashlib.new('ripemd160')
ripmd160.update(Sha256hash_P_bytes)
publickeyhash_bytes = ripmd160.digest()
publickeyhash = ripmd160.hexdigest()
print("Public Key Hash (hex representation): \n",publickeyhash)
print("Public Key Hash (bytes): \n",publickeyhash_bytes)

Public Key Hash (hex representation): 
 a302954979e0a0bc0610afa3c9c2e9e97e73ed93
Public Key Hash (bytes): 
 b'\xa3\x02\x95Iy\xe0\xa0\xbc\x06\x10\xaf\xa3\xc9\xc2\xe9\xe9~s\xed\x93'


### Compute Base58check encoding

In [17]:
!pip install base58 # install base56 Python package if necessary

In [18]:
wprefix_publickeyhash = b'\x00'+publickeyhash_bytes ## prepend prefix byte '00' denoting a P2PKH adress
print(wprefix_publickeyhash)

b'\x00\xa3\x02\x95Iy\xe0\xa0\xbc\x06\x10\xaf\xa3\xc9\xc2\xe9\xe9~s\xed\x93'


In [19]:
import base58
bitcoinaddress = base58.b58encode_check(wprefix_publickeyhash)

In [20]:
print("P2PKH address: \n",bitcoinaddress)

P2PKH address: 
 b'1FrvEkSR2X4KpP7aYHAGZYksfBmRx3t9r9'
